# Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
import plotly.express as px
import requests
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd
import numpy as np
import os
from bs4 import BeautifulSoup
from selenium import webdriver
import matplotlib.pyplot as plt
import re
from rich import print
from rich.progress import track
import seaborn as sns
from sklearn.linear_model import Lasso, Ridge, ElasticNet,LassoCV,RidgeCV
from sklearn.model_selection import cross_val_score, train_test_split,KFold, GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from selenium.webdriver.common.keys import Keys
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import time 


# Import Data

In [ ]:
JawwyTV_df = pd.read_csv("Final_Dataset.txt",sep=",")
JawwyTV_df.sample(3)

In [ ]:
JawwyTV_df.drop(columns="Unnamed: 0",inplace=True) #drop column

In [ ]:
JawwyTV_df.isnull().sum()#Find the null

In [ ]:
all_program = pd.DataFrame(JawwyTV_df[['original_name'].drop_duplicates(), columns=["original_name"])#Take Only the Name

In [ ]:
all_program['url']=''#Add a column for url

In [ ]:
all_program['dec']=''#Add a column for dec

In [ ]:
all_program.reset_index(inplace=True)#Rerange the index

In [ ]:
def get_url(search_term):#scrapping links from imdb
    """Generate url with search term"""
    
    template = "https://www.imdb.com/find?q={}&ref_=nv_sr_sm"#the link for search 
    search_term = search_term.replace(" ", "+")#any space are there in names
    
    
    # Add search term to url
    url = template.format(search_term)

    
    return url

In [ ]:
def extract_record(item):
    """Extract and return data from single record"""

    atag = item.a
    new_url = "https://www.imdb.com"+atag.get('href')#Taking the link for each name

    res = new_url
    return res

In [ ]:
def main(search_term): # this find the link for each Name
    
    chromedriver = "C:\Program Files\Google\Chrome\Application\chromedriver"
    os.environ["webdriver.chrome.driver"] = chromedriver
    DRIVER = webdriver.Chrome(chromedriver)
    RES = None
    URL = get_url(search_term)
    
    for page in range(1):
        DRIVER.get(URL.format(page))
        SOUP = BeautifulSoup(DRIVER.page_source, 'html.parser')
        RESULTS = SOUP.find_all('div', {'class': 'ipc-metadata-list-summary-item__tc'})#Find the div for taking the main link for each name
        for item in RESULTS:
            RES = extract_record(item)
            print(RES)
            if RES:
                break
    
    DRIVER.close()
    
    
    return RES

In [ ]:
def get_data(URL):#this find desc for each link
    chromedriver = "C:\Program Files\Google\Chrome\Application\chromedriver"
    os.environ["webdriver.chrome.driver"] = chromedriver
    DRIVER = webdriver.Chrome(chromedriver)

    dec = None
    for url in track(URL, description='[green]Stealing in progress...'):
        DRIVER.get(url)
        SOUP = BeautifulSoup(DRIVER.page_source, 'html.parser')
        try:
                if SOUP.findAll('div',{ 'class':"sc-16ede01-8 hXeKyz sc-7643a8e3-11 efPxUc"})!=None:#If the div was not there in website will go to find the other one
                    dec = SOUP.find('span',{'class':'sc-16ede01-0 fMPjMP'}).text
                    
                elif  SOUP.findAll('div',{ 'class':"sc-16ede01-7 hrgVKw"})!=None:
                        dec = SOUP.find('span',{'class':'sc-16ede01-0 fMPjMP'}).text
                else:
                       pass
                
        except:
             dec = None
    return dec

In [ ]:
index = 0 #This Function for find the imdb link and find the text description
for i in all_program['original_name']:
    
         for t in range(4):# we put for_loop beacuse some times doesnt extract the text then it will return to find it again 
              all_program.replace(np.nan,'',inplace=True)#To Replace any Nan value to empty string 
              if all_program.loc[index,'url'] == '':
                   url = main(i)
                   all_program.loc[index,'url'] = url
                   if url != None:
                        all_program.loc[index,'dec'] = get_data([url])

              elif all_program.loc[index,'dec'] == ''and all_program.loc[index,'url']!=None :
                        all_program.loc[index,'dec'] = get_data([all_program.loc[index,'url']])
                    

              else: break

         index = index +1

In [ ]:
all_program.to_csv('Get_Desc.csv')#Save DataFrame to CSV 